In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
import warnings; warnings.simplefilter('ignore')

In [2]:
#Import Dataset
md = pd. read_csv('movies data.csv')
md.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [3]:
#Data Preprocessing
md.info()
md.columns
md= md.dropna()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [4]:
# Simple Recommender

In [5]:
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [6]:
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
C

5.828282828282828

In [7]:
m = vote_counts.quantile(0.98)
m

8258.159999999994

In [8]:
md['year'] = pd.to_datetime(md['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [9]:
qualified = md[(md['vote_count'] >= m) & (md['vote_count'].notnull()) & (md['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']]
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.shape

(14, 6)

In [10]:
#Rating depends on Vote Count and Vote Average
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [11]:
qualified['wr'] = qualified.apply(weighted_rating, axis=1)

In [12]:
qualified = qualified.sort_values('wr', ascending=False).head(250)

In [13]:
qualified.head(20)

,title,year,vote_count,vote_average,popularity,genres,wr
12481,The Dark Knight,2008,12269,8,123.167259,"[Drama, Action, Crime, Thriller]",7.126309
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.070725,"[Adventure, Fantasy, Action]",6.954273
14551,Avatar,2009,12114,7,185.070892,"[Action, Adventure, Fantasy, Science Fiction]",6.525027
17818,The Avengers,2012,12000,7,89.887648,"[Science Fiction, Action, Adventure]",6.522354
26564,Deadpool,2016,11444,7,187.860492,"[Action, Adventure, Comedy]",6.508875
23753,Guardians of the Galaxy,2014,10014,7,53.291601,"[Action, Science Fiction, Adventure]",6.470439
26553,Mad Max: Fury Road,2015,9629,7,29.36178,"[Action, Adventure, Science Fiction, Thriller]",6.459041
18252,The Dark Knight Rises,2012,9263,7,20.58258,"[Action, Crime, Drama, Thriller]",6.447740
2458,The Matrix,1999,9079,7,33.366332,"[Action, Science Fiction]",6.441879
12588,Iron Man,2008,8951,7,22.073099,"[Action, Science Fiction, Adventure]",6.437728


In [14]:
s = md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_md = md.drop('genres', axis=1).join(s)

In [15]:
def build_chart(genre, percentile=0.85):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    
    qualified['wr'] = qualified.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(250)
    
    return qualified

In [16]:
#Listing Movies Chart depend on Genre
build_chart('Action').head(15)

,title,year,vote_count,vote_average,popularity,wr
12481,The Dark Knight,2008,12269,8,123.167259,7.390038
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.070725,7.242828
7000,The Lord of the Rings: The Return of the King,2003,8226,8,29.324358,7.204988
5814,The Lord of the Rings: The Two Towers,2002,7641,8,29.423537,7.168487
256,Star Wars,1977,6778,8,42.149697,7.108076
1154,The Empire Strikes Back,1980,5998,8,19.470959,7.045392
14551,Avatar,2009,12114,7,185.070892,6.680839
17818,The Avengers,2012,12000,7,89.887648,6.678711
26564,Deadpool,2016,11444,7,187.860492,6.667914
23753,Guardians of the Galaxy,2014,10014,7,53.291601,6.636495


In [17]:
build_chart('Romance').head(10)

,title,year,vote_count,vote_average,popularity,wr
581,Aladdin,1992,3495,7,16.357419,6.512771
588,Beauty and the Beast,1991,3029,7,23.433511,6.471280
14425,The Twilight Saga: New Moon,2009,2518,5,30.121292,5.401745
18150,The Twilight Saga: Breaking Dawn - Part 1,2011,2622,5,25.9725,5.393496
27478,Fifty Shades of Grey,2015,3350,5,33.068431,5.344048
13133,Twilight,2008,3688,5,31.282029,5.325081


In [18]:
build_chart('Horror').head(20)

,title,year,vote_count,vote_average,popularity,wr
1171,Alien,1979,4564,7,23.37742,6.529499
21276,The Conjuring,2013,3169,7,14.90169,6.391784
1338,Jaws,1975,2628,7,19.726114,6.313904
1888,The Exorcist,1973,2046,7,12.137595,6.204294
39097,The Conjuring 2,2016,2018,7,14.767317,6.198131
2175,The Thing,1982,1629,7,16.83125,6.101439
21130,World War Z,2013,5683,6,26.114917,5.821187
5085,Resident Evil,2002,2100,6,7.781293,5.647888
2058,Blade,1998,1950,6,15.043454,5.632998
18382,Underworld: Awakening,2012,1885,6,12.345348,5.626147


In [19]:
#Content Based Recommender
links = pd.read_csv('links_small.csv')
links = links[links['tmdbId'].notnull()]['tmdbId'].astype('int')

In [20]:
md['id'] = md['id'].astype('int')

In [21]:
smd = md[md['id'].isin(links)]
smd.shape

(464, 25)

In [22]:
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

In [23]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

In [24]:
tfidf_matrix.shape
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim[0]

array([1.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.00705158, 0.00767657, 0.01260097,
       0.01132037, 0.        , 0.        , 0.00846733, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.01029226, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.01412061, 0.        , 0.02144671, 0.        , 0.        ,
       0.0153733 , 0.        , 0.        , 0.        , 0.00981602,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.00826114, 0.        , 0.        , 0.        , 0.        ,
       0.06644677, 0.        , 0.        , 0.        , 0.01141453,
       0.        , 0.        , 0.        , 0.        , 0.04207551,
       0.01546508, 0.0348735 , 0.04231184, 0.06347302, 0.06455093,
       0.08203546, 0.04593027, 0.0142946 , 0.0239397 , 0.     

In [25]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [26]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [27]:
get_recommendations('The Godfather').head(10)

308    Elite Squad: The Enemy Within
47           The Godfather: Part III
292                          Machete
116            House of 1000 Corpses
42                      Halloween II
403                       The Raid 2
262                     Halloween II
437                        Furious 7
172              The Hills Have Eyes
11                        Home Alone
Name: title, dtype: object

In [28]:
get_recommendations('The Dark Knight').head(10)

340                 The Dark Knight Rises
342    Sherlock Holmes: A Game of Shadows
154                         Batman Begins
446    Batman v Superman: Dawn of Justice
382                       Despicable Me 2
40                    Tomorrow Never Dies
104                              Blade II
295       Wall Street: Money Never Sleeps
416          Sin City: A Dame to Kill For
71                            Thunderball
Name: title, dtype: object

In [29]:
credit = pd.read_csv('credits.csv')
keywords = pd.read_csv('keyword.csv')

In [30]:
keywords['id'] = keywords['id'].astype('int')
credit['id'] = credit['id'].astype('int')
md['id'] = md['id'].astype('int')

In [31]:
md.shape

(693, 25)

In [32]:
md = md.merge(credit, on='id')
md = md.merge(keywords, on='id')

In [33]:
smd = md[md['id'].isin(links)]
smd.shape

(472, 28)

In [34]:
smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))

In [35]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [36]:
smd['director'] = smd['crew'].apply(get_director)

In [37]:
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

In [38]:
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [39]:
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [40]:
smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
smd['director'] = smd['director'].apply(lambda x: [x,x, x])

In [41]:
s = smd.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'

In [42]:
s = s.value_counts()
s[:5]
s = s[s > 1]

In [43]:
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

'dog'

In [44]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [45]:
smd['keywords'] = smd['keywords'].apply(filter_keywords)
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [46]:
smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))

In [47]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(smd['soup'])

In [48]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [49]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [50]:
get_recommendations('The Dark Knight').head(10)

347                 The Dark Knight Rises
161                         Batman Begins
453    Batman v Superman: Dawn of Justice
446                      Hitman: Agent 47
459                     London Has Fallen
460                     London Has Fallen
385                          Man of Steel
151                                Pusher
218                           Elite Squad
220                                Hitman
Name: title, dtype: object

In [51]:
#Popularity and Ratings

In [52]:
def improved_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year']]
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified

In [53]:
improved_recommendations('The Dark Knight')

,title,vote_count,vote_average,year,wr
347,The Dark Knight Rises,9263,7,2012,6.447740
161,Batman Begins,7511,7,2005,6.386383
438,Captain America: Civil War,7462,7,2016,6.384470
325,X-Men: First Class,5252,7,2011,6.283781
349,Sherlock Holmes: A Game of Shadows,3971,7,2011,6.208758
158,Sin City,2755,7,2005,6.121394
385,Man of Steel,6462,6,2013,5.903665
384,Fast & Furious 6,5282,6,2013,5.895269
422,Teenage Mutant Ninja Turtles,2677,5,2014,5.625514
453,Batman v Superman: Dawn of Justice,7189,5,2016,5.442806


In [54]:
improved_recommendations('The Godfather')

,title,vote_count,vote_average,year,wr
227,The Dark Knight,12269,8,2008,7.126309
347,The Dark Knight Rises,9263,7,2012,6.447740
161,Batman Begins,7511,7,2005,6.386383
16,Trainspotting,2737,7,1996,6.119956
321,Fast Five,2491,7,2011,6.099815
384,Fast & Furious 6,5282,6,2013,5.895269
380,Jack Reacher,3046,6,2012,5.874553
235,Wanted,2583,6,2008,5.869196
256,Fast & Furious,2426,6,2009,5.867274
186,The Fast and the Furious: Tokyo Drift,1749,6,2006,5.858295


In [55]:
rt = md[md['runtime']== 140]
rt

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,status,tagline,title,video,vote_average,vote_count,year,cast,crew,keywords
166,False,"{'id': 10, 'name': 'Star Wars Collection', 'po...",113000000,"[Science Fiction, Adventure, Action]",http://www.starwars.com/films/star-wars-episod...,1895,tt0121766,en,Star Wars: Episode III - Revenge of the Sith,"Years after the onset of the Clone Wars, the n...",...,Released,The saga is complete.,Star Wars: Episode III - Revenge of the Sith,False,7.1,4200.0,2005,"[{'cast_id': 13, 'character': 'Obi-Wan Kenobi'...","[{'credit_id': '52fe431fc3a36847f803bea3', 'de...","[{'id': 797, 'name': 'showdown'}, {'id': 10013..."
167,False,"{'id': 263, 'name': 'The Dark Knight Collectio...",150000000,"[Action, Crime, Drama]",http://www2.warnerbros.com/batmanbegins/index....,272,tt0372784,en,Batman Begins,"Driven by tragedy, billionaire Bruce Wayne ded...",...,Released,Evil fears the knight.,Batman Begins,False,7.5,7511.0,2005,"[{'cast_id': 13, 'character': 'Bruce Wayne / B...","[{'credit_id': '52fe4230c3a36847f800ac6d', 'de...","[{'id': 486, 'name': 'himalaya'}, {'id': 779, ..."
700,False,"{'id': 173710, 'name': 'Planet of the Apes (Re...",152000000,"[Drama, Science Fiction, War]",http://www.foxmovies.com/movies/war-for-the-pl...,281338,tt3450958,en,War for the Planet of the Apes,Caesar and his apes are forced into a deadly c...,...,Released,For freedom. For family. For the planet.,War for the Planet of the Apes,False,6.7,1675.0,2017,"[{'cast_id': 6, 'character': 'Caesar', 'credit...","[{'credit_id': '591f036fc3a368774e036079', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
